In [1]:
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import urllib.parse as urlparse

from helpers_proc import primaryDomain, extFind, process_date
print(tf.__version__)

1.8.0


### Load data and preprocessing

In [2]:
train_df = pd.read_csv('./train.csv', sep=',')
eval_df = pd.read_csv('./eval.csv', sep=',')
test_df = pd.read_csv('./test.csv', sep=',')

In [3]:
def add_more_features(dataframe):
    df = dataframe.copy()
    
    df['domain_temp'] = ""
    df['domain_temp'] = df.url.apply(lambda row: extFind(row,True))
    df['url_length'] = df.url.apply(lambda row: len(row))
    df['query_string'] = df.url.apply(lambda row: urlparse.urlparse(row).query if urlparse.urlparse(row).query != '' else 'None')
    df['query_length'] = df.query_string.apply(lambda row: len(row))
    df['primary'] = df.domain_temp.apply(lambda row: row[1] if row[0] in ['www','ww1','ww2'] else row[0])
    df['primary_length'] = df.primary.apply(lambda row: len(row))
    df['num_periods'] = df.url.apply(lambda row: row.count("."))
    df['num_exclam'] = df.url.apply(lambda row: row.count("!"))
    df['num_quest'] = df.url.apply(lambda row: row.count("?"))
    df['num_perc'] = df.url.apply(lambda row: row.count("%"))
    df['num_numbers'] = df.url.apply(lambda row: sum(c.isdigit() for c in row))
    df['last_year_modified'] = process_date(df)
    df = df.drop(["query_string","primary","domain_temp"], axis=1)
    return df 

In [4]:
train_df = add_more_features(train_df)
eval_df = add_more_features(eval_df)
test_df = add_more_features(test_df)

In [5]:
train_df.head()

,url,compromissionType,isHiddenFraudulent,contentLength,serverType,poweredBy,contentType,lastModified,url_length,query_length,primary_length,num_periods,num_exclam,num_quest,num_perc,num_numbers,last_year_modified
0,http://etudehouseindo.com/0_themes/etude_jadi/...,phishing,True,0,Apache/2.2,PHP/5.3,text/html; charset=ISO-8859-1,Unknown,113,4,14,1,0,0,0,22,Unknown
1,http://islanderthemovie.com/scripts/alibaba.html,phishing,True,15464,Apache,Unknown,text/html,"Tue, 13 Nov 2012 07:47:07 GMT",48,4,16,2,0,0,0,0,2012
2,http://bjcurio.com/js/index.htm?http://us.batt...,phishing,True,4422,Microsoft-IIS/6.0,Unknown,text/html,"Sun, 11 Nov 2012 04:47:06 GMT",133,101,7,6,0,2,6,8,2012
3,http://www.acqf.info/page/vote.php?pid=64,phishing,False,1,Apache/2.2,PHP/5.2,text/html; charset=UTF-8,Unknown,41,6,9,3,0,1,0,2,Unknown
4,http://www.protect-effect.nl/component/user/re...,defacement,False,0,Apache,Unknown,text/html; charset=utf-8,"Tue, 29 Jan 2013 18:10:29 GMT",50,4,14,2,0,0,0,0,2013


In [6]:
def get_vocabulary_list(category):
    train_cat = list(train_df[category].unique())
    eval_cat = list(eval_df[category].unique())
    test_cat = list(test_df[category].unique())
    return sorted(set(train_cat + eval_cat + test_cat))

In [7]:
len(get_vocabulary_list('serverType'))

128

In [8]:
# Create pandas input function
def make_train_input_fn(df, batch_size, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x = df,
        y = df['isHiddenFraudulent'],
        batch_size = batch_size,
        num_epochs = num_epochs,
        shuffle = True,
        queue_capacity = 1024,
        num_threads = 1
    )

def make_eval_input_fn(df, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = df,
        y = df['isHiddenFraudulent'], 
        batch_size = 256,
        num_epochs = 1,
        shuffle = False,
        queue_capacity = 1024,
        num_threads = 1
    )

def make_test_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x = df,
        batch_size = 16,
        num_epochs = 1,
        shuffle = False,
        queue_capacity = 1024,
        num_threads = 1
    )

#### Features to take into account

- query_length / done
- primary_length
- num_periods
- num_quest
- num_perc
- num_numbers

In [9]:
num_int_features = ['url_length', 'contentLength', 'query_length', 'num_perc', 'primary_length']
category_features = ['serverType', 'poweredBy', 'contentType', 'last_year_modified']

In [10]:
def create_feature_cols(embedding=False, dnn=True):
    
    tf_fc_emb = tf.feature_column.embedding_column
    tf_fc_num = tf.feature_column.numeric_column
    tf_fc_cat = tf.feature_column.categorical_column_with_vocabulary_list
    tf_fc_ind = tf.feature_column.indicator_column
    
    num_cols = [tf_fc_num(col) for col in num_int_features]
    cat_cols = [tf_fc_cat(key=col, 
                          vocabulary_list=get_vocabulary_list(col)) for col in category_features]
    if embedding:
        cat_cols = [tf_fc_emb(col, 16) for col in cat_cols]
    if dnn:
        cat_cols = [tf_fc_ind(col) for col in cat_cols]
    cols = num_cols + cat_cols
    
    #try bucketize url_length 5 
    return cols 

feature_cols = create_feature_cols(embedding=False, dnn=True)

In [11]:
def serving_input_fn():
    
    n_int = len(num_int_features)
    n_cat = len(category_features)
    
    num_placeholders = [tf.placeholder(tf.int64, [None]) for i in range(n_int)]
    string_placeholders = [tf.placeholder(tf.string, [None]) for i in range(n_cat)]
    
    feat_names = num_int_features + category_features
    placeholders = num_placeholders + string_placeholders
    
    json_feature_placeholders = dict(zip(feat_names, placeholders))
    
    features = json_feature_placeholders
    
    return tf.estimator.export.ServingInputReceiver(features, json_feature_placeholders)

In [46]:
# def serving_input_fn():
    
#     def get_length(single_string):
#         x = tf.expand_dims(single_string,0)
#         splitted = tf.string_split(x, delimiter="").values
#         length = tf.size(splitted)
#         return length

#     def get_lengths(string_batch):
#         return tf.map_fn(get_length, string_batch, dtype=tf.int32)
    
#     json_feature_placeholders = {
#         'url': tf.placeholder(tf.string, None),
#     }
    
#     features = {
#         'url_length': get_lengths(json_feature_placeholders['url'])
#     }    

#     return tf.estimator.export.ServingInputReceiver(features, json_feature_placeholders)

### Each model function

In [22]:
train_df.shape[0]//BATCH_SIZE * NUM_EPOCH

11920

In [101]:
# Create estimator train and evaluate function
NUM_EPOCH = 20
BATCH_SIZE = 256
SAVE_CKPT_STEPS = 200 # steps
EVAL_INTERVAL_SEC =  5 # sec
HIDDEN_UNITS = [256,128,64,32]
MAX_STEPS = train_df.shape[0]//BATCH_SIZE * NUM_EPOCH
LEARNING_RATE_LINEAR = 0.008
LEARNING_RATE_DNN = 0.0008
DROPOUT = 0.05
def train_and_evaluate(output_dir, model='linear'):
    
    run_config = tf.estimator.RunConfig(model_dir=output_dir, 
                                        save_summary_steps=50,
                                        keep_checkpoint_max=10,
                                        save_checkpoints_steps=SAVE_CKPT_STEPS)
    
    if model == 'linear':
#         optimizer = tf.train.FtrlOptimizer(learning_rate=lr)
        estimator = tf.estimator.LinearClassifier(feature_columns=feature_cols, 
                                                   optimizer=tf.train.FtrlOptimizer(learning_rate=LEARNING_RATE_LINEAR),
                                                  config=run_config)
    else:
#         optimizer = tf.train.ProximalAdagradOptimizer(learning_rate=LEARNING_RATE_DNN)
        optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE_DNN)
        
        estimator = tf.estimator.DNNClassifier(feature_columns=feature_cols, 
                                               hidden_units=HIDDEN_UNITS,
                                               optimizer=optimizer,
                                               config=run_config,
                                               dropout=DROPOUT)

    train_spec = tf.estimator.TrainSpec(input_fn=make_train_input_fn(train_df, BATCH_SIZE, NUM_EPOCH), 
                                        max_steps=MAX_STEPS)

    export_latest = tf.estimator.LatestExporter(name='exporter', 
                                                serving_input_receiver_fn=serving_input_fn,
                                                exports_to_keep=None)

    eval_spec = tf.estimator.EvalSpec(input_fn=make_eval_input_fn(eval_df, BATCH_SIZE), 
                                    steps=None,
                                    start_delay_secs = 1, # start evaluating after N seconds
                                    throttle_secs = EVAL_INTERVAL_SEC,     # evaluate every N seconds
                                    exporters=export_latest
                                    )

    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    
    return estimator 

In [102]:
# parameters_name = ['NUM_EPOCH','BATCH_SIZE','SAVE_CKPT_STEPS','EVAL_INTERVAL_SEC','HIDDEN_UNITS','MAX_STEPS']
# parameters_value = [20, 256, 200, 5, [32,8,2], train_df.shape[0]//BATCH_SIZE * NUM_EPOCH]
# params = dict(zip(parameters_name,parameters_value))

In [103]:
outdir = '../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/'

In [104]:
shutil.rmtree(outdir, ignore_errors = True)
estimator = train_and_evaluate(outdir, model='dnn')

INFO:tensorflow:Using config: {'_model_dir': '../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12fed8c50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 5 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHoo

INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'num_perc': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=int64>, 'primary_length': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_6:0' shape=(?,) dtype=string>, 'contentType': <tf.Tensor 'Placeholder_7:0' shape=(?,) dtype=string>, 'last_year_modified': <tf.Tensor 'Placeholder_8:0' shape=(?,) dtype=string>}
INFO:tensorflow:'classification' : Classification input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'content

INFO:tensorflow:loss = 81.943085, step = 998
INFO:tensorflow:global_step/sec: 70.5683
INFO:tensorflow:loss = 71.82105, step = 1098 (1.418 sec)
INFO:tensorflow:Saving checkpoints for 1198 into ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt.
INFO:tensorflow:global_step/sec: 95.6348
INFO:tensorflow:loss = 74.47606, step = 1198 (1.045 sec)
INFO:tensorflow:global_step/sec: 87.6402
INFO:tensorflow:loss = 66.02815, step = 1298 (1.142 sec)
INFO:tensorflow:Saving checkpoints for 1339 into ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt.
INFO:tensorflow:Loss for final step: 69.40303.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-14-16:55:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt-1339
INFO:tensorflow:Running local_init_op.
INFO:tenso

INFO:tensorflow:'regression' : Regression input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'num_perc': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=int64>, 'primary_length': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_6:0' shape=(?,) dtype=string>, 'contentType': <tf.Tensor 'Placeholder_7:0' shape=(?,) dtype=string>, 'last_year_modified': <tf.Tensor 'Placeholder_8:0' shape=(?,) dtype=string>}
INFO:tensorflow:Restoring parameters from ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt-1658
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"../trained_model/a_ql_pl_nq_dnn_256_128_

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'num_perc': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=int64>, 'primary_length': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_6:0' shape=(?,) dtype=string>, 'contentType': <tf.Tensor 'Placeholder_7:

...

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-14-16:59:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt-10105
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-14-16:59:44
INFO:tensorflow:Saving dict for global step 10105: accuracy = 0.8552868, accuracy_baseline = 0.93881804, auc = 0.9409438, auc_precision_recall = 0.69706434, average_loss = 0.2067065, global_step = 10105, label/mean = 0.061181948, loss = 52.638702, precision = 0.287021, prediction/mean = 0.15259321, recall = 0.9199683
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']


INFO:tensorflow:Restoring parameters from ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt-10467
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/export/exporter/temp-b'1552582792'/saved_model.pb"
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt-10467
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10468 into ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt.
INFO:tensorflow:loss = 59.54864, step = 10468
INFO:tensorflow:global_step/sec: 68.815
INFO:tensorflow:loss = 58.990257, step = 1

INFO:tensorflow:'classification' : Classification input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'num_perc': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=int64>, 'primary_length': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_6:0' shape=(?,) dtype=string>, 'contentType': <tf.Tensor 'Placeholder_7:0' shape=(?,) dtype=string>, 'last_year_modified': <tf.Tensor 'Placeholder_8:0' shape=(?,) dtype=string>}
INFO:tensorflow:'regression' : Regression input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) 

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-14-17:00:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt-11911
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-14-17:00:25
INFO:tensorflow:Saving dict for global step 11911: accuracy = 0.8525234, accuracy_baseline = 0.93881804, auc = 0.9425776, auc_precision_recall = 0.6974523, average_loss = 0.20805393, global_step = 11911, label/mean = 0.061181948, loss = 52.98183, precision = 0.2843992, prediction/mean = 0.15548141, recall = 0.9302694
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']


INFO:tensorflow:Restoring parameters from ../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/model.ckpt-11920
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"../trained_model/a_ql_pl_nq_dnn_256_128_64_32_lr_8e4_adam_dropout_0_0_5/export/exporter/temp-b'1552582830'/saved_model.pb"


### LR study

In [41]:
lrs = np.linspace(0.001,0.1,15)

In [42]:
for lr in lrs:
    outdir = '../learning_rate_study/{}'.format(lr)
    shutil.rmtree(outdir, ignore_errors = True)
    estimator = train_and_evaluate(outdir, lr, model='linear')

INFO:tensorflow:Using config: {'_model_dir': '../learning_rate_study/0.001', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1331a7e10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 5 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

INFO:tensorflow:'classification' : Classification input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=string>, 'contentType': <tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=string>, 'last_year_modified': <tf.Tensor 'Placeholder_6:0' shape=(?,) dtype=string>}
INFO:tensorflow:'regression' : Regression input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_4:0' shape=(?,) dt

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=string>, 'contentType': <tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=string>, 'last_year_modified': <tf.Tensor 'Placeholder_6:0' shape=(?,) dtype=string>}
INFO:tensorflow:'classification'

INFO:tensorflow:global_step/sec: 179.053
INFO:tensorflow:loss = 117.66867, step = 3189 (0.559 sec)
INFO:tensorflow:Saving checkpoints for 3223 into ../learning_rate_study/0.001/model.ckpt.
INFO:tensorflow:Loss for final step: 122.2244.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-14-12:19:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../learning_rate_study/0.001/model.ckpt-3223
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-14-12:19:11
INFO:tensorflow:Saving dict for global step 3223: accuracy = 0.879042, accuracy_baseline = 0.93881804, auc = 0.86533195, auc_precision_recall = 0.38222882, average_loss = 0.42414936, global_step = 3223, label/mean = 0.061181948, loss = 108.01147, precision = 0.2655002, prediction/mean = 0.33353424, recall = 0.55309033
INFO:tensorflow:Calling model_fn.
INFO:tensorf

INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"../learning_rate_study/0.001/export/exporter/temp-b'1552565959'/saved_model.pb"
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../learning_rate_study/0.001/model.ckpt-3823
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3824 into ../learning_rate_study/0.001/model.ckpt.
INFO:tensorflow:loss = 125.724556, step = 3824
INFO:tensorflow:global_step/sec: 92.2385
INFO:tensorflow:loss = 110.58783, step = 3924 (1.085 sec)
INFO:tensorflow:Saving checkpoints for 4024 into ../learning_rate_study/0.001/model.ckpt.
INFO:tensorflow:global_step/sec: 157.945
INFO:tensorflow:loss = 114.16916, step = 4024 (0.633 sec)
INFO:tensorflow:global_step/sec: 183.456
INFO:tensorflow:loss = 117.51349, step = 4124 (0.546 

...

INFO:tensorflow:Restoring parameters from ../learning_rate_study/0.1/model.ckpt-8030
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"../learning_rate_study/0.1/export/exporter/temp-b'1552570604'/saved_model.pb"
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../learning_rate_study/0.1/model.ckpt-8030
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 8031 into ../learning_rate_study/0.1/model.ckpt.
INFO:tensorflow:loss = 62.725758, step = 8031
INFO:tensorflow:global_step/sec: 97.7726
INFO:tensorflow:loss = 130.31761, step = 8131 (1.024 sec)
INFO:tensorflow:Saving checkpoints for 8231 into ../learning_rate_study/0.1/model.ckpt.
INFO:tensorflow:global_step/sec: 156.234
INFO:tensorflow:loss = 443.35922, ste

INFO:tensorflow:'classification' : Classification input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=string>, 'contentType': <tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=string>, 'last_year_modified': <tf.Tensor 'Placeholder_6:0' shape=(?,) dtype=string>}
INFO:tensorflow:'regression' : Regression input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_4:0' shape=(?,) dt

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'url_length': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'contentLength': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'query_length': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'serverType': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=string>, 'poweredBy': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=string>, 'contentType': <tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=string>, 'last_year_modified': <tf.Tensor 'Placeholder_6:0' shape=(?,) dtype=string>}
INFO:tensorflow:'classification'

INFO:tensorflow:global_step/sec: 163.302
INFO:tensorflow:loss = 1079.9138, step = 11406 (0.613 sec)
INFO:tensorflow:Saving checkpoints for 11411 into ../learning_rate_study/0.1/model.ckpt.
INFO:tensorflow:Loss for final step: 129.69267.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-14-13:37:33
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../learning_rate_study/0.1/model.ckpt-11411
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-14-13:37:34
INFO:tensorflow:Saving dict for global step 11411: accuracy = 0.9287342, accuracy_baseline = 0.93881804, auc = 0.9254034, auc_precision_recall = 0.6981839, average_loss = 0.27124023, global_step = 11411, label/mean = 0.061181948, loss = 69.072495, precision = 0.45126522, prediction/mean = 0.12276658, recall = 0.76307446
INFO:tensorflow:Calling model_fn.
INFO:tenso

INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"../learning_rate_study/0.1/export/exporter/temp-b'1552570662'/saved_model.pb"


### Serving 

In [99]:
url_ex = "https://github.com/warmspringwinds/tf-image-segmentation/issues/10"
len(url_ex)

66

In [100]:
%%writefile url.json
{"url_lenght": 66}

Overwriting url.json


In [ ]:
%%bash

location=/Users/paul/Desktop/Google_ASL/trained_model/url_len_dnn/export/exporter/1552430670
gcloud ml-engine local predict --model-dir=$location --json-instances=url.json --verbosity debug 

#### Load models

In [109]:
import tensorflow as tf

output_dir = '../trained_model/len_linear/'

def create_feature_cols():
    cols = [tf.feature_column.numeric_column('url_length')]
    #try bucketize url_length 5 
    return cols 

feature_cols = create_feature_cols()

run_config = tf.estimator.RunConfig(model_dir=output_dir, 
                                  keep_checkpoint_max=10,
                                  save_summary_steps=100, 
                                  save_checkpoints_steps=200)

estimator = tf.estimator.LinearClassifier(feature_columns=feature_cols, 
                                       config=run_config)

INFO:tensorflow:Using config: {'_model_dir': '../trained_model/len_linear/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x15bcf4a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [110]:
predictions = estimator.predict(input_fn=make_test_input_fn(test_df))

In [111]:
list(predictions)[:20]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../trained_model/len_linear/model.ckpt-7150
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'logits': array([-0.13137773], dtype=float32),
  'logistic': array([0.46720272], dtype=float32),
  'probabilities': array([0.5327973 , 0.46720272], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([0.10907033], dtype=float32),
  'logistic': array([0.5272406], dtype=float32),
  'probabilities': array([0.47275943, 0.5272406 ], dtype=float32),
  'class_ids': array([1]),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([0.10907033], dtype=float32),
  'logistic': array([0.5272406], dtype=float32),
  'probabilities': array([0.47275943, 0.5272406 ], dtype=float32),
  'class_ids': array([1]),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([-0.22221364], dtype=float32),
  'logistic': array([0.44467404], dtype=float32),
  'probabilities': array([0.5553259 , 0.44467407], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([0.09838375], dtype=float32),
  'logistic'

In [108]:
pwd

'/Users/paul/Desktop/Google_ASL/fraud-url'